In [1]:
import numpy as np
import tensorflow as tf
import pickle
import os
from tensorflow.keras.preprocessing.sequence import pad_sequences

### Demo

In [2]:
GRU_MODEL_PATH = 'BiGRU_W2V_best.keras'
LSTM_MODEL_PATH = 'BiLSTM_W2V_best.keras'
TOKENIZER_PATH = 'tokenizer.pickle'
MAX_SEQUENCE_LENGTH = 300

In [3]:
try:
    gru_model = tf.keras.models.load_model(GRU_MODEL_PATH)
    lstm_model = tf.keras.models.load_model(LSTM_MODEL_PATH)
    print("Deep Learning model loaded successfully.")
    gru_model.summary()
    lstm_model.summary()
except Exception as e:
    print(f"Error loading Deep Learning model: {e}")
    print("Please ensure the model file exists and is compatible.")

Deep Learning model loaded successfully.


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ (None, 300, 100)       │       292,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_8 (Bidirectional) │ (None, 300, 192)       │       114,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_9 (Bidirectional) │ (None, 96)             │        69,696 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 96)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 32)             │         3,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 2)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,437,644 (5.48 MB)

 Trainable params: 479,214 (1.83 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ (None, 300, 100)       │       292,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_6 (Bidirectional) │ (None, 300, 192)       │       151,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_7 (Bidirectional) │ (None, 96)             │        92,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 96)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 32)             │         3,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 2)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,617,932 (6.17 MB)

 Trainable params: 539,310 (2.06 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,078,622 (4.11 MB)

In [4]:
# Load the tokenizer from TOKENIZER_PATH
with open(TOKENIZER_PATH, 'rb') as handle:
    tokenizer = pickle.load(handle)

In [5]:
import sys
sys.path.insert(0, "../../")

In [6]:
from src.shared.text_preprocessing import VietnamesePreprocessingWithoutSpellCheck

Current file path: D:\Documents\Ki_8\NLP\Comment-Sentiment-Analysis
Thu mục gốc: D:\Documents\Ki_8\NLP\Comment-Sentiment-Analysis
Using model path: D:\Documents\Ki_8\NLP\Comment-Sentiment-Analysis\attribute_extractor\vncorenlp


In [7]:
without_spellcheck_preprocessor = VietnamesePreprocessingWithoutSpellCheck()

In [8]:
def predict_sentiment(text, model, tokenizer, max_length):
    # clean_text = text # Tạm thời bỏ qua làm sạch phức tạp
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=max_length, padding='post', truncating='post')
    prediction = model.predict(padded_sequence, verbose=0) # Thêm verbose=0
    labels = (prediction > 0.5).astype(int)[0]
    print(f"Predicted labels (positive, negative): {labels}")
    print(f"Probabilities:")
    print(f"P(positive=1): {prediction[0][0]}")
    print(f"P(negative=1): {prediction[0][1]}")
    predicted_dict = {"positive": int(labels[0]), "negative": int(labels[1])}
    return predicted_dict, prediction[0]

In [ ]:
# Nhập input liên tục từ người dùng cho đến khi người dùng nhập "exit"
while True:
    user_input = input("Nhập một comment hoặc 'exit' để thoát: ")
    
    if user_input.lower() == 'exit':
        print("Thoát chương trình.")
        break
    
    print("==================================")
    print("Comment gốc:", user_input)
    proceeded_input = without_spellcheck_preprocessor(user_input)
    print("Comment:", proceeded_input)
    pred_labels, pred_probs = predict_sentiment(proceeded_input, gru_model, tokenizer, MAX_SEQUENCE_LENGTH)

Comment gốc: xin chào
Comment: chào
Predicted labels (positive, negative): [1 0]
Probabilities:
P(positive=1): 0.996173083782196
P(negative=1): 0.00766832334920764
Comment gốc: sản phẩm tốt
Comment: sản_phẩm tốt
Predicted labels (positive, negative): [1 0]
Probabilities:
P(positive=1): 0.997830331325531
P(negative=1): 0.000977963674813509
Comment gốc: pin nhanh nóng quá
Comment: pin nhanh nóng quá
Predicted labels (positive, negative): [0 1]
Probabilities:
P(positive=1): 0.02740609273314476
P(negative=1): 0.9339203834533691
Comment gốc: máy mượt, chiến game mạnh
Comment: máy mượt chiến game mạnh
Predicted labels (positive, negative): [1 0]
Probabilities:
P(positive=1): 0.9158605337142944
P(negative=1): 0.18729235231876373
Comment gốc: tháng này bên hãng có khuyến mãi gì không
Comment: tháng bên hãng khuyến_mãi gì không
Predicted labels (positive, negative): [0 0]
Probabilities:
P(positive=1): 0.0010322934249415994
P(negative=1): 0.04406847432255745
Comment gốc: nhân viên khó chịu sao ấ

## Test GRU

In [9]:
# Nhập input liên tục từ người dùng cho đến khi người dùng nhập "exit"
while True:
    user_input = input("Nhập một comment hoặc 'exit' để thoát: ")
    
    if user_input.lower() == 'exit':
        print("Thoát chương trình.")
        break
    
    print("==================================")
    print("Comment gốc:", user_input)
    proceeded_input = without_spellcheck_preprocessor(user_input)
    print("Comment:", proceeded_input)
    pred_labels, pred_probs = predict_sentiment(proceeded_input, gru_model, tokenizer, MAX_SEQUENCE_LENGTH)

Comment gốc: xin chào
Comment: chào
Predicted labels (positive, negative): [1 0]
Probabilities:
P(positive=1): 0.9975305199623108
P(negative=1): 0.0006178136100061238
Comment gốc: sản phẩm tốt
Comment: sản_phẩm tốt
Predicted labels (positive, negative): [1 0]
Probabilities:
P(positive=1): 0.9992673397064209
P(negative=1): 0.0009938040748238564
Comment gốc: pin nhanh nóng quá
Comment: pin nhanh nóng quá
Predicted labels (positive, negative): [0 1]
Probabilities:
P(positive=1): 0.10131531208753586
P(negative=1): 0.967193603515625
Comment gốc: máy mượt, chiến game mạnh
Comment: máy mượt chiến game mạnh
Predicted labels (positive, negative): [1 0]
Probabilities:
P(positive=1): 0.9993622899055481
P(negative=1): 0.0004375790013000369
Comment gốc: tháng này bên hãng có khuyến mãi gì không
Comment: tháng bên hãng khuyến_mãi gì không
Predicted labels (positive, negative): [0 0]
Probabilities:
P(positive=1): 0.0008991212234832346
P(negative=1): 0.0011965448502451181
Comment gốc: nhân viên khó ch

## Test LSTM


In [10]:
# Nhập input liên tục từ người dùng cho đến khi người dùng nhập "exit"
while True:
    user_input = input("Nhập một comment hoặc 'exit' để thoát: ")
    
    if user_input.lower() == 'exit':
        print("Thoát chương trình.")
        break
    
    print("==================================")
    print("Comment gốc:", user_input)
    proceeded_input = without_spellcheck_preprocessor(user_input)
    print("Comment:", proceeded_input)
    pred_labels, pred_probs = predict_sentiment(user_input, lstm_model, tokenizer, MAX_SEQUENCE_LENGTH)

Comment gốc: xin chào
Comment: chào
Predicted labels (positive, negative): [1 0]
Probabilities:
P(positive=1): 0.91893070936203
P(negative=1): 0.022992543876171112
Comment gốc: sản phẩm tốt
Comment: sản_phẩm tốt
Predicted labels (positive, negative): [1 0]
Probabilities:
P(positive=1): 0.9918392896652222
P(negative=1): 0.005109638907015324
Comment gốc: pin nhanh nóng quá
Comment: pin nhanh nóng quá
Predicted labels (positive, negative): [0 1]
Probabilities:
P(positive=1): 0.05473865568637848
P(negative=1): 0.9679790139198303
Comment gốc: máy mượt, chiến game mạnh
Comment: máy mượt chiến game mạnh
Predicted labels (positive, negative): [1 0]
Probabilities:
P(positive=1): 0.9950771927833557
P(negative=1): 0.0025364840403199196
Comment gốc: tháng này bên hãng có khuyến mãi gì không
Comment: tháng bên hãng khuyến_mãi gì không
Predicted labels (positive, negative): [0 0]
Probabilities:
P(positive=1): 0.0016965708928182721
P(negative=1): 0.00144206325057894
Comment gốc: nhân viên khó chịu sa

In [12]:
# Nhập input liên tục từ người dùng cho đến khi người dùng nhập "exit"
while True:
    user_input = input("Nhập một comment hoặc 'exit' để thoát: ")
    
    if user_input.lower() == 'exit':
        print("Thoát chương trình.")
        break
    
    print("==================================")
    print("Comment gốc:", user_input)
    proceeded_input = without_spellcheck_preprocessor(user_input)
    print("Comment:", proceeded_input)
    pred_labels, pred_probs = predict_sentiment(user_input, gru_model, tokenizer, MAX_SEQUENCE_LENGTH)

Comment gốc: tôi sẽ không bây giờ quay lại cái cửa hàng này nữa
Comment: không quay lại cửa_hàng
Predicted labels (positive, negative): [0 1]
Probabilities:
P(positive=1): 0.00922361109405756
P(negative=1): 0.9797496795654297
Thoát chương trình.
